# Project (Section 5)

This is where we left off in the previous section project:

In [25]:
from datetime import date
from enum import Enum
from pydantic import BaseModel, ConfigDict, Field, field_serializer
from pydantic.alias_generators import to_camel


class AutomobileType(Enum):
    sedan = "Sedan"
    coupe = "Coupe"
    convertible = "Convertible"
    suv = "SUV"
    truck = "Truck"


class Automobile(BaseModel):
    model_config = ConfigDict(
        extra="forbid",
        str_strip_whitespace=True,
        validate_default=True,
        validate_assignment=True,
        alias_generator=to_camel,
    )
    
    manufacturer: str
    series_name: str
    type_: AutomobileType = Field(alias="type")
    is_electric: bool = False
    manufactured_date: date = Field(validation_alias="completionDate")
    base_msrp_usd: float = Field(
        validation_alias="msrpUSD", 
        serialization_alias="baseMSRPUSD"
    )
    vin: str
    number_of_doors: int = Field(default=4, validation_alias="doors")
    registration_country: str | None = None
    license_plate: str | None = None

    @field_serializer("manufactured_date", when_used="json-unless-none")
    def serialize_date(self, value: date) -> str:
        return value.strftime("%Y/%m/%d")

Modify your `Automobile` model to implement the following:
- add a field named `id_`
    - make it the **first** field in your model
    - if provided in the data, it will be named `id`
    - it should deserialize to `id`
    - the field type should be a uuid4
    - for now, have it default to `None`

You can use this data to test your model:

In [26]:
from datetime import date
from enum import Enum
from pydantic import BaseModel, ConfigDict, Field, field_serializer, UUID4
from pydantic.alias_generators import to_camel
from uuid import uuid4


class AutomobileType(Enum):
    sedan = "Sedan"
    coupe = "Coupe"
    convertible = "Convertible"
    suv = "SUV"
    truck = "Truck"


class Automobile(BaseModel):
    model_config = ConfigDict(
        extra="forbid",
        str_strip_whitespace=True,
        validate_default=True,
        validate_assignment=True,
        alias_generator=to_camel,
    )
    id_: UUID4 | None = Field(alias="id", default=None)
    manufacturer: str
    series_name: str
    type_: AutomobileType = Field(alias="type")
    is_electric: bool = False
    manufactured_date: date = Field(validation_alias="completionDate")
    base_msrp_usd: float = Field(
        validation_alias="msrpUSD", 
        serialization_alias="baseMSRPUSD"
    )
    vin: str
    number_of_doors: int = Field(default=4, validation_alias="doors")
    registration_country: str | None = None
    license_plate: str | None = None

    @field_serializer("manufactured_date", when_used="json-unless-none")
    def serialize_date(self, value: date) -> str:
        return value.strftime("%Y/%m/%d")

In [27]:
data = {
    "id": "c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7",
    "manufacturer": "BMW",
    "seriesName": "M4",
    "type": "Convertible",
    "isElectric": False,
    "completionDate": "2023-01-01",
    "msrpUSD": 93_300,
    "vin": "1234567890",
    "doors": 2,
    "registrationCountry": "France",
    "licensePlate": "AAA-BBB"
}

Expected serialization to dict by alias is:

In [28]:
from uuid import UUID

expected_serialized_by_alias = {
    'id': UUID('c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7'),
    'manufacturer': 'BMW',
    'seriesName': 'M4',
    'type': AutomobileType.convertible,
    'isElectric': False,
    'manufacturedDate': date(2023, 1, 1),
    'baseMSRPUSD': 93300.0,
    'vin': '1234567890',
    'numberOfDoors': 2,
    'registrationCountry': 'France',
    'licensePlate': 'AAA-BBB'
}

In [29]:
car = Automobile.model_validate(data)
car

Automobile(id_=UUID('c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7'), manufacturer='BMW', series_name='M4', type_=<AutomobileType.convertible: 'Convertible'>, is_electric=False, manufactured_date=datetime.date(2023, 1, 1), base_msrp_usd=93300.0, vin='1234567890', number_of_doors=2, registration_country='France', license_plate='AAA-BBB')

In [30]:
assert car.model_dump(by_alias=True) == expected_serialized_by_alias 

In [31]:
data_no_id = {
    "manufacturer": "BMW",
    "seriesName": "M4",
    "type": "Convertible",
    "isElectric": False,
    "completionDate": "2023-01-01",
    "msrpUSD": 93_300,
    "vin": "1234567890",
    "doors": 2,
    "registrationCountry": "France",
    "licensePlate": "AAA-BBB"
}

In [32]:
expected_serialization_data_no_id_by_alias = {
    'id': None,
    'manufacturer': 'BMW',
    'seriesName': 'M4',
    'type': AutomobileType.convertible,
    'isElectric': False,
    'manufacturedDate': date(2023, 1, 1),
    'baseMSRPUSD': 93300.0,
    'vin': '1234567890',
    'numberOfDoors': 2,
    'registrationCountry': 'France',
    'licensePlate': 'AAA-BBB'
}

In [33]:
car = Automobile.model_validate(data_no_id)
car.model_fields

/var/folders/36/92m48z6515nbv9cpxw0sm4y40000gn/T/ipykernel_43615/1984275745.py:2: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  car.model_fields


{'id_': FieldInfo(annotation=Union[Annotated[UUID, UuidVersion], NoneType], required=False, default=None, alias='id', alias_priority=2),
 'manufacturer': FieldInfo(annotation=str, required=True, alias='manufacturer', alias_priority=1),
 'series_name': FieldInfo(annotation=str, required=True, alias='seriesName', alias_priority=1),
 'type_': FieldInfo(annotation=AutomobileType, required=True, alias='type', alias_priority=2),
 'is_electric': FieldInfo(annotation=bool, required=False, default=False, alias='isElectric', alias_priority=1),
 'manufactured_date': FieldInfo(annotation=date, required=True, alias='manufacturedDate', alias_priority=2, validation_alias='completionDate'),
 'base_msrp_usd': FieldInfo(annotation=float, required=True, alias='baseMsrpUsd', alias_priority=2, validation_alias='msrpUSD', serialization_alias='baseMSRPUSD'),
 'vin': FieldInfo(annotation=str, required=True, alias='vin', alias_priority=1),
 'number_of_doors': FieldInfo(annotation=int, required=False, default=4

In [34]:
assert car.model_dump(by_alias=True) == expected_serialization_data_no_id_by_alias